# Дистилляция [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer) (50 баллов)

Будем дистиллировать [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer) для [задачи сегментации людей](https://www.kaggle.com/datasets/laurentmih/aisegmentcom-matting-human-datasets).

## Скачаем вспомогательный код и чекпоинт бейзлайна (модели-учителя)

In [ ]:
!wget -O hw_files.zip 'https://www.dropbox.com/scl/fi/jmnz7nsf72ou13tgaxfat/hw_01_dist_files.zip?rlkey=e6cfsgfumdx60pc9sza1wggbb&dl=0'
!unzip hw_files.zip

### Скачаем датасет

Датасет находится по ссылке https://disk.yandex.ru/d/iBF7MQVMWAZk2A

Нужно его скачать и распаковать в папке, в которой находится ноутбук

### Установим библиотеки

In [ ]:
!pip install torch transformers datasets tensorboard pillow

In [1]:
import os

import typing as tp
import torch

from copy import deepcopy
from datasets import load_metric
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard.writer import SummaryWriter
from tqdm.auto import tqdm

# utils у нас появились при скачивании вспомогательного кода. При желании можно в них провалиться-поизучать
from utils.data import init_dataloaders
from utils.model import evaluate_model
from utils.model import init_model_with_pretrain

from torch import nn
from transformers.models.segformer.modeling_segformer import SegformerLayer
from transformers import SegformerForSemanticSegmentation

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/gvasserm/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
teacher_path = '../runs/baseline_ckpt.pth'
save_dir = '../runs/distillation/'

In [3]:
tb_writer = SummaryWriter(save_dir)

In [4]:
# маппинг названия классов и индексов
id2label = {
    0: "background",
    1: "human",
}
label2id = {v: k for k, v in id2label.items()}

Создадим лоадеры:

In [5]:
train_dataloader, valid_dataloader = init_dataloaders(
    root_dir="/home/gvasserm/data/matting_human_dataset/",
    batch_size=8,
    num_workers=8,
)

/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


Создадим модель учителя:

In [6]:
teacher_model = init_model_with_pretrain(label2id=label2id, id2label=id2label, pretrain_path=teacher_path)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


И сразу отвалидируем:

In [7]:
evaluate_model(teacher_model, valid_dataloader, id2label)

/home/gvasserm/dev/ml_acceleration/assignment1/src/utils/model.py:38: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("mean_iou")
/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.9825463313870122
Mean accuracy: 0.9910212045247916


{'mean_iou': 0.9825463313870122,
 'mean_accuracy': 0.9910212045247916,
 'overall_accuracy': 0.9912375183105469,
 'per_category_iou': array([0.98135781, 0.98373485]),
 'per_category_accuracy': array([0.98774413, 0.99429828])}

## Делаем ученика (10 баллов)

### Посмотрим, как выглядит модель учителя:

In [8]:
teacher_model

SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(160, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  

Нас интересует (block): он состоит из нескольких ModuleList. Нас интересуют первые четыре. Посмотрим на первый из них:

```
(0): ModuleList(
          (0): SegformerLayer(
            .... тут много понаписано
          )
          (1): SegformerLayer(
            .... и тут тоже много всего
        )
```

В каждом из четырёх ModuleList сидит по два `SegformerLayer`. Нужно написать функцию, которая оставит только один (последний) из них.

In [9]:
def create_small_network(model):
    model.config.depths = [1,1,1,1]
    small_model = SegformerForSemanticSegmentation(model.config)

    devices = torch.device("cuda:0")
    state_dict = model.state_dict()

    new_state_dict = {}
    for k, v in state_dict.items():
        if 'segformer.encoder.block' in k:
            if k.split('.')[4] == '0':
                continue
            else:
                new_state_dict[k] = v
        else:
            new_state_dict[k] = v

    small_model.load_state_dict(new_state_dict, strict=False)
 
    return small_model.cuda()
    
    return small_model

def n_params(model):
    return sum(p.numel() for p in model.parameters())

In [10]:
student_model = create_small_network(deepcopy(teacher_model))

In [ ]:
# визуализируйте и убедитесь, что у вас действительно выкинуты нужные слои
student_model

In [11]:
print(f'Teacher model size: {n_params(teacher_model)}')
print(f'Student model size: {n_params(student_model)}')

Teacher model size: 3714658
Student model size: 2298210


## Train Loop

Напишем старый-добрый трейнлуп и добавим в него дистилляционные лоссы.

In [12]:
from dataclasses import dataclass

@dataclass
class TrainParams:
    n_epochs: int
    lr: float
    batch_size: int
    n_workers: int
    device: torch.device

    loss_weight: float
    last_layer_loss_weight: float
    intermediate_attn_layers_weights: tp.Tuple[float, float, float, float]
    intermediate_feat_layers_weights: tp.Tuple[float, float, float, float]
    temperature: float

    # возможно, в ваших экспериментах захотите добавить что-то ещё

#### Домашка на поиграться-поэкспериментировать, поэтому не стесняйтесь менять параметры и выбивать скоры

In [13]:
train_params = TrainParams(
    n_epochs=4,
    lr=6e-5,
    batch_size=8,
    n_workers=8,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    loss_weight=0.5,
    last_layer_loss_weight=0.5,
    intermediate_attn_layers_weights=(0.5, 0.5, 0.5, 0.5),
    intermediate_feat_layers_weights=(0.5, 0.5, 0.5, 0.5),
    temperature=3.0
)

In [14]:
def calc_last_layer_loss(student_logits, teacher_logits, weight, temperature=1.0):
    
    student_probs = F.log_softmax(student_logits / temperature, dim=1)
    teacher_probs = F.softmax(teacher_logits / temperature, dim=1)

    kl_divergence_loss = nn.KLDivLoss(reduction='batchmean')
    loss = kl_divergence_loss(student_probs, teacher_probs) * (temperature ** 2) * weight

    return loss
    
# здесь пока не обращаем внимания, чуть позже её напишем
def calc_intermediate_layers_attn_loss(student_logits, teacher_logits, weights, student_teacher_attention_mapping):
    return None

# здесь пока не обращаем внимания, чуть позже её напишем
def calc_intermediate_layers_feat_loss(student_feat, teacher_feat, weights):
    return None

In [15]:
def train(
    teacher_model,
    student_model,
    train_params: TrainParams,
    student_teacher_attention_mapping,
):
    metric = load_metric('mean_iou')
    teacher_model.to(train_params.device)
    student_model.to(train_params.device)

    teacher_model.eval()

    train_dataloader, valid_dataloader = init_dataloaders(
        root_dir="/home/gvasserm/data/matting_human_dataset/",
        batch_size=train_params.batch_size,
        num_workers=train_params.n_workers,
    )

    optimizer = torch.optim.AdamW(student_model.parameters(), lr=train_params.lr)
    step = 0
    for epoch in range(train_params.n_epochs):
        pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        for idx, batch in pbar:
            student_model.train()
            # get the inputs;
            pixel_values = batch['pixel_values'].to(train_params.device)
            labels = batch['labels'].to(train_params.device)

            optimizer.zero_grad()

            # forward + backward + optimize
            student_outputs = student_model(
                pixel_values=pixel_values, 
                labels=labels, 
                output_attentions=True,
                output_hidden_states=True,
            )
            loss, student_logits = student_outputs.loss, student_outputs.logits

            # Чего это мы no_grad() при тренировке поставили?!
            with torch.no_grad():
                teacher_output = teacher_model(
                    pixel_values=pixel_values, 
                    labels=labels, 
                    output_attentions=True,
                    output_hidden_states=True,
                )


            last_layer_loss = calc_last_layer_loss(
                student_logits,
                teacher_output.logits,
                train_params.last_layer_loss_weight,
                temperature=train_params.temperature
            )

            student_attentions, teacher_attentions = student_outputs.attentions, teacher_output.attentions
            student_hidden_states, teacher_hidden_states = student_outputs.hidden_states, teacher_output.hidden_states

            intermediate_layer_att_loss = calc_intermediate_layers_attn_loss(
                student_attentions,
                teacher_attentions,
                train_params.intermediate_attn_layers_weights,
                student_teacher_attention_mapping,
            )
            
            intermediate_layer_feat_loss = calc_intermediate_layers_feat_loss(
                student_hidden_states,
                teacher_hidden_states,
                train_params.intermediate_feat_layers_weights,
            )

            total_loss = loss* train_params.loss_weight + last_layer_loss
            if intermediate_layer_att_loss is not None:
                total_loss += intermediate_layer_att_loss
            
            if intermediate_layer_feat_loss is not None:
                total_loss += intermediate_layer_feat_loss

            step += 1

            total_loss.backward()
            optimizer.step()
            pbar.set_description(f'total loss: {total_loss.item():.3f}')

            for loss_value, loss_name in (
                (loss, 'loss'),
                (total_loss, 'total_loss'),
                (last_layer_loss, 'last_layer_loss'),
                (intermediate_layer_att_loss, 'intermediate_layer_att_loss'),
                (intermediate_layer_feat_loss, 'intermediate_layer_feat_loss'),
            ):
                if loss_value is None: # для выключенной дистилляции атеншенов
                    continue
                tb_writer.add_scalar(
                    tag=loss_name,
                    scalar_value=loss_value.item(),
                    global_step=step,
                )

        #после модификаций модели обязательно сохраняйте ее целиком, чтобы подгрузить ее в случае чего
        torch.save(
            {
                'model': student_model,
                'state_dict': student_model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
            },
            f'{save_dir}/ckpt_{epoch}.pth',
        )

        eval_metrics = evaluate_model(student_model, valid_dataloader, id2label)

        for metric_key, metric_value in eval_metrics.items():
            if not isinstance(metric_value, float):
                continue
            tb_writer.add_scalar(
                tag=f'eval_{metric_key}',
                scalar_value=metric_value,
                global_step=epoch,
            )


### Включим-посмотрим, как учится

In [16]:
train(
    teacher_model=teacher_model,
    student_model=deepcopy(student_model),
    train_params=train_params,
    student_teacher_attention_mapping={}, # заполним потом
)

/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is de

### Лосс для дистилляции атеншн-мап (20б)

Из каждого сегформер-блока можно достать атеншн-мапы:

In [ ]:
with torch.no_grad():
    teacher_attentions = teacher_model(pixel_values=torch.ones(1, 3, 512, 512).to(train_params.device), output_attentions=True).attentions
    student_attentions = student_model(pixel_values=torch.ones(1, 3, 512, 512).to(train_params.device), output_attentions=True).attentions

In [ ]:
teacher_attentions[0].shape

In [ ]:
assert len(teacher_attentions) == 8
assert len(student_attentions) == 4

Будем дистиллировать и их!
Но у учителя у нас их целых 8, а у ученика четыре. Поэтому нужно сделать соответствие: номер какой фичемапы у ученика
будем тянуть к какому номеру фичемапы учителя.

Сделайте правильное соответствие **(10б)**:

In [ ]:
student_teacher_attention_mapping = {i: i*2 + 1 for i in range(4)}

Теперь напишите лосс, который принимает на вход списки фичемап ученика и учителя и тянет одно к другому. Возможно, вы захотите
учитывать разные фичемапы с разными весами. Для этого воспользуйтесь `weights` **10б**

### Лосс для дистилляции промежуточных фиче-мап (10б)

Помимо внимания, у вас также есть карты признаков, которые можно стягивать. Напишите лосс, который бы стягивал их. Вполне возможно, что стоит их стягивать с весами, и вполне возможно что эти веса будутотличаться от весов для внимания.

In [ ]:
#здесь пока не обращаем внимания, чуть позже её напишем
def calc_intermediate_layers_attn_loss(student_logits, teacher_logits, weights, student_teacher_attention_mapping):
    total_loss = 0.0
    for student_idx, teacher_idx in student_teacher_attention_mapping.items():
        student_att = student_logits[student_idx]
        teacher_att = teacher_logits[teacher_idx]
        
        loss = F.mse_loss(student_att, teacher_att)
        
        # Apply weight
        weighted_loss = loss * weights[student_idx]
        
        # Accumulate loss
        total_loss += weighted_loss
    
    return total_loss

# здесь пока не обращаем внимания, чуть позже её напишем
def calc_intermediate_layers_feat_loss(student_feat, teacher_feat, weights):
    total_loss = 0.0
    assert len(student_feat) == len(teacher_feat) == len(weights), "Mismatch in the number of layers or weights"
    
    for student_feat, teacher_feat, weight in zip(student_feat, teacher_feat, weights):
        loss = F.mse_loss(student_feat, teacher_feat)
        total_loss += loss * weight
    
    return total_loss

### Теперь можем тренировать со стягиванием разных фич 

In [ ]:
def init_model_student_with_pretrain(pretrain_path):
    
    devices = torch.device("cuda:0")
    params = torch.load(pretrain_path, devices)
    model = SegformerForSemanticSegmentation(params['model'].config)
    model.load_state_dict(params['state_dict'], strict=False)

    return model.cuda()

student_model = init_model_student_with_pretrain("assignment1/runs/distillation/ckpt_3.pth")
train_params.n_epochs = 3
train(
    teacher_model=teacher_model,
    student_model=deepcopy(student_model),
    train_params=train_params,
    student_teacher_attention_mapping=student_teacher_attention_mapping,
)

## Отправка решения

Загрузите ноутбук на образовательную платформу. Если удалось плотно поэкспериментировать — расскажите, что зашло и докинула ли вам дистилляция промежуточных слоёв

Model       Size	BS	LR	       SSLoss KDLoss AttLoss AttIntLoss	    T 	    E	       Mean_iou	Mean_acc
Teacher	    3714658									                                        0.9825	0.991
Student	    2298210	24	1.20E-04	0.5	  0.5	0	0	                1.2	    3	        0.922	0.959
Student 	2298210	24	1.20E-04	0.5	  0.5	0	0	                1.05	3	        0.916	0.9562
Student 	2298210	8	6.00E-05	0.5	  0.5	0	0	                1.2	    3	        0.9217	0.9589
Student 	2298210	24	1.60E-04	0.5	  0.5	0	0	                3	    4	        0.931	0.9639
Student 	2298210	24	1.80E-04	0.5	  0.5	0	0	                3	    4	        0.93346	0.9655
Student 	2298210	24	1.80E-04	0.5	  0.5	0	0	                3	    5	        0.941677	0.9696
Student 	2298210	24	1.80E-04	0.5	  0.5	0.5	0.5	                3	    4+3	        0.9668	0.983